In [ ]:
#hide
from fastai.vision.all import *
from utils import *

matplotlib.rc('image', cmap='Greys')

## Magia konwolucji

In [ ]:
top_edge = tensor([[-1,-1,-1],
                   [ 0, 0, 0],
                   [ 1, 1, 1]]).float()

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
im3 = Image.open(path/'train'/'3'/'12.png')
show_image(im3);

In [ ]:
im3_t = tensor(im3)
im3_t[0:3,0:3] * top_edge

In [ ]:
(im3_t[0:3,0:3] * top_edge).sum()

In [ ]:
df = pd.DataFrame(im3_t[:10,:20])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [ ]:
(im3_t[4:7,6:9] * top_edge).sum()

In [ ]:
(im3_t[7:10,17:20] * top_edge).sum()

In [ ]:
def apply_kernel(row, col, kernel):
    return (im3_t[row-1:row+2,col-1:col+2] * kernel).sum()

In [ ]:
apply_kernel(5,7,top_edge)

### Odwzorowywanie jądra splotu

In [ ]:
[[(i,j) for j in range(1,5)] for i in range(1,5)]

In [ ]:
rng = range(1,27)
top_edge3 = tensor([[apply_kernel(i,j,top_edge) for j in rng] for i in rng])

show_image(top_edge3);

In [ ]:
left_edge = tensor([[-1,1,0],
                    [-1,1,0],
                    [-1,1,0]]).float()

left_edge3 = tensor([[apply_kernel(i,j,left_edge) for j in rng] for i in rng])

show_image(left_edge3);

### Konwolucje w bibliotece PyTorch

In [ ]:
diag1_edge = tensor([[ 0,-1, 1],
                     [-1, 1, 0],
                     [ 1, 0, 0]]).float()
diag2_edge = tensor([[ 1,-1, 0],
                     [ 0, 1,-1],
                     [ 0, 0, 1]]).float()

edge_kernels = torch.stack([left_edge, top_edge, diag1_edge, diag2_edge])
edge_kernels.shape

In [ ]:
mnist = DataBlock((ImageBlock(cls=PILImageBW), CategoryBlock), 
                  get_items=get_image_files, 
                  splitter=GrandparentSplitter(),
                  get_y=parent_label)

dls = mnist.dataloaders(path)
xb,yb = first(dls.valid)
xb.shape

In [ ]:
xb,yb = to_cpu(xb),to_cpu(yb)

In [ ]:
edge_kernels.shape,edge_kernels.unsqueeze(1).shape

In [ ]:
edge_kernels = edge_kernels.unsqueeze(1)

In [ ]:
batch_features = F.conv2d(xb, edge_kernels)
batch_features.shape

In [ ]:
show_image(batch_features[0,0]);

### Tworzenie konwolucyjnej sieci neuronowej

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [ ]:
simple_net

In [ ]:
broken_cnn = sequential(
    nn.Conv2d(1,30, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(30,1, kernel_size=3, padding=1)
)

In [ ]:
broken_cnn(xb).shape

In [ ]:
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

In [ ]:
simple_cnn = sequential(
    conv(1 ,4),            #14x14
    conv(4 ,8),            #7x7
    conv(8 ,16),           #4x4
    conv(16,32),           #2x2
    conv(32,2, act=False), #1x1
    Flatten(),
)

In [ ]:
simple_cnn(xb).shape

In [ ]:
learn = Learner(dls, simple_cnn, loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(2, 0.01)

### Zrozumienie arytmetyki konwolucji

In [ ]:
m = learn.model[0]
m

In [ ]:
m[0].weight.shape

In [ ]:
m[0].bias.shape

## Obrazy kolorowe

In [ ]:
im = image2tensor(Image.open('images/grizzly.jpg'))
im.shape

In [ ]:
show_image(im);

In [ ]:
_,axs = subplots(1,3)
for bear,ax,color in zip(im,axs,('Reds','Greens','Blues')):
    show_image(255-bear, ax=ax, cmap=color)

## Ulepszanie stabilności trenowania

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
def get_dls(bs=64):
    return DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock), 
        get_items=get_image_files, 
        splitter=GrandparentSplitter('training','testing'),
        get_y=parent_label,
        batch_tfms=Normalize()
    ).dataloaders(path, bs=bs)

dls = get_dls()

In [ ]:
dls.show_batch(max_n=9, figsize=(4,4))

### Prosty model bazowy

In [ ]:
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

In [ ]:
def simple_cnn():
    return sequential(
        conv(1 ,8, ks=5),        #14x14
        conv(8 ,16),             #7x7
        conv(16,32),             #4x4
        conv(32,64),             #2x2
        conv(64,10, act=False),  #1x1
        Flatten(),
    )

In [ ]:
from fastai.callback.hook import *

In [ ]:
def fit(epochs=1):
    learn = Learner(dls, simple_cnn(), loss_func=F.cross_entropy,
                    metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit(epochs, 0.06)
    return learn

In [ ]:
learn = fit()

In [ ]:
learn.activation_stats.plot_layer_stats(0)

In [ ]:
learn.activation_stats.plot_layer_stats(-2)

### Zwiększenie wielkości paczki

In [ ]:
dls = get_dls(512)

In [ ]:
learn = fit()

In [ ]:
learn.activation_stats.plot_layer_stats(-2)

### Trenowanie jednocykliczne

In [ ]:
def fit(epochs=1, lr=0.06):
    learn = Learner(dls, simple_cnn(), loss_func=F.cross_entropy,
                    metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit_one_cycle(epochs, lr)
    return learn

In [ ]:
learn = fit()

In [ ]:
learn.recorder.plot_sched()

In [ ]:
learn.activation_stats.plot_layer_stats(-2)

In [ ]:
learn.activation_stats.color_dim(-2)

In [ ]:
learn.activation_stats.color_dim(-2)

### Normalizacja wsadowa

In [ ]:
def conv(ni, nf, ks=3, act=True):
    layers = [nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)]
    layers.append(nn.BatchNorm2d(nf))
    if act: layers.append(nn.ReLU())
    return nn.Sequential(*layers)

In [ ]:
learn = fit()

In [ ]:
learn.activation_stats.color_dim(-4)

In [ ]:
learn = fit(5, lr=0.1)

In [ ]:
learn = fit(5, lr=0.1)